# 2023년 데이터마이닝 기출문제 - 완성 버전
## 문제 1: Spotify 데이터 회귀 분석 (45점)

### 필요한 라이브러리 import

In [ ]:
import pandas as pd
import numpy as np
import math
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

### 데이터 로딩 및 확인

In [ ]:
spotify_df = pd.read_csv('spotify-2023-2.csv')
print("[데이터 정보]")
spotify_df.info()

print("\n[결측치 확인]")
print(spotify_df.isnull().sum())

print("\n[데이터 샘플]")
print(spotify_df.head())

### (1) 독립변수/종속변수 분리

In [ ]:
# 범주형 변수 변환
spotify_df['released_year'] = spotify_df['released_year'].astype('category')
spotify_df['released_month'] = spotify_df['released_month'].astype('category')

# 종속변수: streams
outcome = 'streams'

# 독립변수: streams, key, track_name, artist(s)_name, in_spotify_charts 제외
predictors = [col for col in spotify_df.columns 
              if col not in ['streams', 'key', 'track_name', 'artist(s)_name', 'in_spotify_charts']]

X = spotify_df[predictors]
y = spotify_df[outcome]

# 원-핫 인코딩
X = pd.get_dummies(X, drop_first=False)

print(f"독립변수 개수: {X.shape[1]}")
print(f"데이터 크기: {X.shape}")
print(f"\n변수 목록 (처음 10개):\n{X.columns.tolist()[:10]}")

### (2) 학습/테스트 데이터 분리 (0~899: 학습, 900~: 테스트)

In [ ]:
# 학습: 0~899행, 테스트: 900행~
X_train = X.iloc[:900]
y_train = y.iloc[:900]
X_test = X.iloc[900:]
y_test = y.iloc[900:]

print(f"총 데이터 수: {len(X)}")
print(f"학습 데이터 (Train): {len(X_train)} 행")
print(f"테스트 데이터 (Test): {len(X_test)} 행")

### (3) 각 모델 생성 및 RMSE 측정

In [ ]:
results = {}

# 1) 선형회귀
print("[1] 선형회귀 (Linear Regression)")
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
lr_pred = lr_model.predict(X_test)
lr_mse = mean_squared_error(y_test, lr_pred)
lr_rmse = math.sqrt(lr_mse)
results['Linear Regression'] = lr_rmse
print(f"   RMSE: {lr_rmse:,.2f}")

In [ ]:
# 2) Ridge
print("[2] Ridge Regression")
ridge_model = Ridge(alpha=1.0, random_state=42)
ridge_model.fit(X_train, y_train)
ridge_pred = ridge_model.predict(X_test)
ridge_mse = mean_squared_error(y_test, ridge_pred)
ridge_rmse = math.sqrt(ridge_mse)
results['Ridge'] = ridge_rmse
print(f"   RMSE: {ridge_rmse:,.2f}")

In [ ]:
# 3) Lasso
print("[3] Lasso Regression")
lasso_model = Lasso(alpha=1.0, random_state=42, max_iter=10000)
lasso_model.fit(X_train, y_train)
lasso_pred = lasso_model.predict(X_test)
lasso_mse = mean_squared_error(y_test, lasso_pred)
lasso_rmse = math.sqrt(lasso_mse)
results['Lasso'] = lasso_rmse
print(f"   RMSE: {lasso_rmse:,.2f}")

In [ ]:
# 4) ElasticNet
print("[4] ElasticNet Regression")
elastic_model = ElasticNet(alpha=1.0, l1_ratio=0.5, random_state=42, max_iter=10000)
elastic_model.fit(X_train, y_train)
elastic_pred = elastic_model.predict(X_test)
elastic_mse = mean_squared_error(y_test, elastic_pred)
elastic_rmse = math.sqrt(elastic_mse)
results['ElasticNet'] = elastic_rmse
print(f"   RMSE: {elastic_rmse:,.2f}")

In [ ]:
# 모델별 RMSE 요약
print("\n[모델별 RMSE 요약]")
for model_name, rmse in results.items():
    print(f"  {model_name:20s}: {rmse:,.2f}")

### (4) 하이퍼파라미터 튜닝 - Lasso 최적화

In [ ]:
# Lasso 튜닝 (GridSearchCV)
lasso_tuner = Lasso(random_state=42, max_iter=10000)

param_grid = {
    'alpha': np.logspace(-4, 0, 50)  # 0.0001부터 1.0까지
}

grid_search = GridSearchCV(
    lasso_tuner,
    param_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=0
)

print("튜닝 진행 중...")
grid_search.fit(X_train, y_train)

best_alpha = grid_search.best_params_['alpha']
best_mse = -grid_search.best_score_
best_rmse = math.sqrt(best_mse)

print(f"\n**[튜닝 결과]**")
print(f"최적의 Alpha: {best_alpha:.6f}")
print(f"최고 CV RMSE: {best_rmse:,.2f}")

# 최적 모델로 테스트 데이터 예측
best_model = grid_search.best_estimator_
best_pred = best_model.predict(X_test)
best_test_mse = mean_squared_error(y_test, best_pred)
best_test_rmse = math.sqrt(best_test_mse)

print(f"\n**[최종 성과]**")
print(f"최적 튜닝 모델의 Test RMSE: {best_test_rmse:,.2f}")

# 0이 아닌 계수 확인
coef_df = pd.DataFrame({
    'Variable': X.columns,
    'Coefficient': best_model.coef_
})
non_zero = coef_df[coef_df['Coefficient'] != 0]
print(f"\n0이 아닌 계수 개수: {len(non_zero)} / {len(coef_df)}")

---
## 문제 2: Airline 만족도 분류 (50점)

### 필요한 라이브러리 import

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split

### 데이터 로딩 및 확인

In [ ]:
airline_df = pd.read_csv('airline.csv')
print("[데이터 정보]")
airline_df.info()

print("\n[결측치 확인]")
print(airline_df.isnull().sum())

print("\n[데이터 샘플]")
print(airline_df.head())

### (1) 불필요한 변수 제거
예: ID, Unnamed 등 불필요한 컬럼 제거

### (2) 독립변수/종속변수 분리 및 원-핫 인코딩

In [ ]:
outcome = 'Satisfaction'
predictors = [col for col in airline_df.columns if col not in ['Satisfaction']]

X = airline_df[predictors]
y = airline_df[outcome]

# 원-핫 인코딩
X = pd.get_dummies(X, drop_first=False)

print(f"독립변수 개수: {X.shape[1]}")
print(f"데이터 크기: {X.shape}")

# 학습/검증 데이터 분리 (70:30)
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.3, random_state=1, stratify=y
)

print(f"\n학습 데이터: {len(X_train)} 행")
print(f"검증 데이터: {len(X_valid)} 행")

### (3) 의사결정나무 모델 생성 및 5-Fold CV (AUC)

In [ ]:
tree_full = DecisionTreeClassifier(criterion='entropy', random_state=1)
tree_full.fit(X_train, y_train)

# 5-Fold 교차검증
scores = cross_val_score(tree_full, X, y, cv=5, scoring='roc_auc')
print(f"5-Fold CV ROC AUC: {scores.mean():.4f} (±{scores.std():.4f})")

### (4) max_depth 최적값 찾기

In [ ]:
depths = range(2, 21)
cv_scores = []

for depth in depths:
    tree = DecisionTreeClassifier(max_depth=depth, criterion='entropy', random_state=1)
    scores = cross_val_score(tree, X, y, cv=5, scoring='roc_auc')
    cv_scores.append(scores.mean())

# 최적 depth 찾기
best_depth_idx = np.argmax(cv_scores)
best_depth = list(depths)[best_depth_idx]
best_score = cv_scores[best_depth_idx]

print(f"최적 max_depth: {best_depth}")
print(f"최고 CV AUC: {best_score:.4f}")

### (5) 최적 depth로 모델 생성 및 5-Fold CV

In [ ]:
tree_optimal = DecisionTreeClassifier(
    criterion='entropy',
    max_depth=best_depth,
    random_state=1
)
tree_optimal.fit(X_train, y_train)

# 5-Fold 교차검증
scores_optimal = cross_val_score(tree_optimal, X, y, cv=5, scoring='roc_auc')
print(f"5-Fold CV ROC AUC: {scores_optimal.mean():.4f} (±{scores_optimal.std():.4f})")

### (6) 변수 중요도 확인 및 Top 10 선택

In [ ]:
importance = tree_optimal.feature_importances_
feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importance
}).sort_values(by='Importance', ascending=False)

print("[상위 10개 중요 변수]")
top_10_features = feature_importance_df.head(10)['Feature'].tolist()
print(feature_importance_df.head(10))

### (7) 로지스틱 회귀 - Top 10 변수만 사용

In [ ]:
logreg_model = LogisticRegression(
    random_state=42,
    solver='liblinear',
    max_iter=1000
)

# 5-Fold 교차검증
auc_scores_logreg = cross_val_score(
    logreg_model,
    X[top_10_features],
    y,
    cv=5,
    scoring='roc_auc'
)

print(f"[로지스틱 회귀 (Top 10 변수) 5-Fold AUC]: {auc_scores_logreg.mean():.4f}")

### (8) 모델 성과 비교

In [ ]:
print("\n[모델 성과 비교]")
print(f"의사결정나무 (최적 depth={best_depth}): {scores_optimal.mean():.4f}")
print(f"로지스틱 회귀 (Top 10 변수):          {auc_scores_logreg.mean():.4f}")

if scores_optimal.mean() > auc_scores_logreg.mean():
    print(f"\n결론: 의사결정나무가 더 우수한 성과를 보임 (차이: {scores_optimal.mean() - auc_scores_logreg.mean():.4f})")
else:
    print(f"\n결론: 로지스틱 회귀가 더 우수한 성과를 보임 (차이: {auc_scores_logreg.mean() - scores_optimal.mean():.4f})")